In [1]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
db_path = "yelp_data.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Get all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [table[0] for table in cursor.fetchall()]

print("Tables in the database:", tables)

conn.close()


Tables in the database: ['business', 'review', 'amazon_reviews', 'sqlite_sequence', 'imdb_reviews']


In [4]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy

# Download necessary NLP resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Load spaCy model for lemmatization
nlp = spacy.load("en_core_web_sm")
lemmatizer = WordNetLemmatizer()

# Preprocessing function
def preprocess_text(text):
    if not isinstance(text, str):  # Ensure input is a string
        return ""
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)

    # Remove irrelevant characters (numbers, punctuation)
    text = re.sub(r'[^A-Za-z\s]', '', text)

    # Convert to lowercase
    text = text.lower()

    # Tokenization
    tokens = nltk.word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Apply Lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Remove short words (length ≤ 2)
    tokens = [word for word in tokens if len(word) > 2]

    # Convert tokens back to string
    return ' '.join(tokens)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\smrit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\smrit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\smrit\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
def preprocess_all_tables(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Fetch all table names
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [table[0] for table in cursor.fetchall()]

    for table in tables:
        print(f"Processing table: {table}")

        # Get column names for the table
        cursor.execute(f"PRAGMA table_info({table});")
        columns = [col[1] for col in cursor.fetchall()]

        # Find the first TEXT column (assuming reviews are stored there)
        text_column = None
        for col in columns:
            cursor.execute(f"SELECT typeof({col}) FROM {table} LIMIT 5;")
            types = {row[0] for row in cursor.fetchall()}
            if "text" in types or "TEXT" in types:
                text_column = col
                break

        if not text_column:
            print(f" No text column found in {table}. Skipping...")
            continue

        # Load data into DataFrame
        df = pd.read_sql(f"SELECT * FROM {table}", conn)

        # Check for missing values and drop rows with missing text
        df.dropna(subset=[text_column], inplace=True)

        # Apply preprocessing
        df['preprocessed_text'] = df[text_column].apply(preprocess_text)

        # Save back to the database
        df.to_sql(table, conn, if_exists='replace', index=False)

        print(f" Preprocessed and updated {table}.")

    conn.close()
    print(" All tables processed!")

# Run the function
preprocess_all_tables("yelp_data.db")


Processing table: business
 Preprocessed and updated business.
Processing table: review
 Preprocessed and updated review.
Processing table: amazon_reviews
 Preprocessed and updated amazon_reviews.
Processing table: sqlite_sequence
 No text column found in sqlite_sequence. Skipping...
Processing table: imdb_reviews
 Preprocessed and updated imdb_reviews.
 All tables processed!


In [6]:
import sqlite3
import pandas as pd

def preview_preprocessed_data(db_path, table_name):
    """Fetch and display a sample of preprocessed data."""
    conn = sqlite3.connect(db_path)

    # Load the data from the given table
    df = pd.read_sql(f"SELECT * FROM {table_name}", conn)

    conn.close()

    # Show sample data
    print(" Sample Preprocessed Data:")
    print(df[['preprocessed_text']].sample(5))  # Display 5 random rows




In [7]:
# Example Usage
db_path = "yelp_data.db"
table_name = "business"  
preview_preprocessed_data(db_path, table_name)

 Sample Preprocessed Data:
           preprocessed_text
129717    yovumkpubntdyawzvg
8591      ptlgcnxyfhmpiyenbq
139515    esbkcxfpzytzpatilq
23378   wvokxjymnvuasfnvlopw
130850   wqiuqccpmfkhxhchgsw


In [8]:
# Example Usage
db_path = "yelp_data.db"
table_name = "review"  
preview_preprocessed_data(db_path, table_name)

 Sample Preprocessed Data:
          preprocessed_text
3979515    bkfyqtaaruqqvgkw
2296209  dssmrqbzgrknnkxpew
3681452  dbgtgkqfljrixhdgag
5540758    wnycovttglrtwbiq
964497    hmeefrochrgyzhilq


In [9]:
# Example Usage
db_path = "yelp_data.db"
table_name = "imdb_reviews"  
preview_preprocessed_data(db_path, table_name)

 Sample Preprocessed Data:
                                       preprocessed_text
23376  great documentary life firefighter worst terro...
20371  first review saw page said madhur bhandarkar f...
7337   well let say always steven seagal fan movie us...
9102   movie epitomizes fear even today fear people p...
18205  wonderland spoiler july five people ron launiu...


In [10]:
# Example Usage
db_path = "yelp_data.db"
table_name = "amazon_reviews"  
preview_preprocessed_data(db_path, table_name)

 Sample Preprocessed Data:
                  preprocessed_text
2167085                   fan first
2813913             great price job
977808   polished design unique spa
3368766                  misleading
2184207               shipping much


In [11]:
import sqlite3

# Connect to SQLite database
db_path = "yelp_data.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Fetch column names from the review table
cursor.execute("PRAGMA table_info(review);")  # Get table schema
columns = cursor.fetchall()

# Print column names
print(" Columns in 'review' table:")
for col in columns:
    print(f"- {col[1]}")  # Column name is at index 1

conn.close()


 Columns in 'review' table:
- review_id
- business_id
- stars
- text
- date
- preprocessed_text


In [12]:
import sqlite3
import pandas as pd

# Connect to SQLite database
db_path = "yelp_data.db"
conn = sqlite3.connect(db_path)

# Fetch sample data
query = "SELECT review_id, stars, text, preprocessed_text FROM review LIMIT 5;"
df = pd.read_sql_query(query, conn)

# Display data
print(df)

# Close connection
conn.close()


                review_id  stars  \
0  KU_O5udG6zpxOg-VcAEodg    3.0   
1  BiTunyQ73aT9WBnpR9DZGw    5.0   
2  saUsX_uimxRlCVr67Z4Jig    3.0   
3  AqPFMleE6RsU23_auESxiA    5.0   
4  Sx8TMOWLNuJBWer-0pcmoA    4.0   

                                                text    preprocessed_text  
0  If you decide to eat here, just be aware it is...   kuoudgzpxogvcaeodg  
1  I've taken a lot of spin classes over the year...   bitunyqatwbnprdzgw  
2  Family diner. Had the buffet. Eclectic assortm...   sausxuimxrlcvrzjig  
3  Wow!  Yummy, different,  delicious.   Our favo...   aqpfmleersuauesxia  
4  Cute interior and owner (?) gave us tour of up...  sxtmowlnujbwerpcmoa  


In [ ]:
from nltk.tokenize import word_tokenize

# Connect to SQLite database
db_path = "yelp_data.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Step 1: Add `preprocessed_text` column if not exists
cursor.execute("PRAGMA table_info(review);")
columns = [col[1] for col in cursor.fetchall()]
if "preprocessed_text" not in columns:
    cursor.execute("ALTER TABLE review ADD COLUMN preprocessed_text TEXT;")
    conn.commit()

# Preprocessing function
def preprocess_text(text):
    if not text:
        return ""

    # Convert to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)

    # Remove numbers and punctuation
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenization
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Remove words with length ≤ 2
    tokens = [word for word in tokens if len(word) > 2]

    # Convert list of tokens back to string
    return " ".join(tokens)

# Fetch all reviews from the table
cursor.execute("SELECT review_id, text FROM review")
rows = cursor.fetchall()

# # Step 2: Preprocess and update the database
# for review_id, text in rows:
#     preprocessed_text = preprocess_text(text)
#     cursor.execute("UPDATE review SET preprocessed_text = ? WHERE review_id = ?", (preprocessed_text, review_id))

# # Commit changes and close connection
# conn.commit()
# conn.close()

# print(" Preprocessing complete! `preprocessed_text` column updated in `review` table.")


In [4]:
import json
import pandas as pd
import spacy

# Load SpaCy model (disable unnecessary components for speed)
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

def preprocess_text(text):
    """Preprocess text by lemmatizing and filtering stopwords."""
    doc = nlp(text)
    return " ".join(token.lemma_ for token in doc if token.is_alpha and not token.is_stop and len(token.text) > 2)

# Load the GoEmotions dataset (JSON file)
with open('goemotions_cleaned.json', 'r') as file:
    goemotions_data = json.load(file)

# Convert to DataFrame
goemotions_df = pd.DataFrame(goemotions_data)

# Apply preprocessing
goemotions_df['preprocessed_text'] = goemotions_df['text'].apply(preprocess_text)

# Save back to JSON file
with open('goemotions_cleaned.json', 'w') as file:
    json.dump(goemotions_df.to_dict(orient="records"), file, indent=4)

print("Preprocessing complete. File updated with preprocessed text.")


Preprocessing complete. File updated with preprocessed text.


In [6]:
import sqlite3
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Download required NLTK data
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def preprocess_text(text):
    if not text:
        return ""

    # Convert to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)

    # Remove numbers and punctuation
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenization
    tokens = word_tokenize(text)

    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Remove words with length ≤ 2
    tokens = [word for word in tokens if len(word) > 2]

    # Convert list of tokens back to string
    return " ".join(tokens)

# Connect to SQLite database
conn = sqlite3.connect("yelp_data.db")

# Fetch a random sample of 100K reviews for fast preprocessing
query = "SELECT review_id, business_id,stars,date, text FROM review ORDER BY RANDOM() LIMIT 100000"
df_sample = pd.read_sql(query, conn)
query1 = "SELECT * FROM review LIMIT 2"
df_sample1 = pd.read_sql(query1, conn)
conn.close()
print(df_sample1)

# Apply preprocessing
df_sample["preprocessed_text"] = df_sample["text"].apply(preprocess_text)

# Save the preprocessed sample to a CSV or JSON file
df_sample.to_csv("yelp_sample_preprocessed.csv", index=False)
df_sample.to_json("yelp_sample_preprocessed.json", orient="records", indent=4)

print("Preprocessing completed! Saved the preprocessed Yelp sample.")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\smrit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\smrit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\smrit\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                review_id             business_id  stars  \
0  KU_O5udG6zpxOg-VcAEodg  XQfwVwDr-v0ZS3_CbbE5Xw    3.0   
1  BiTunyQ73aT9WBnpR9DZGw  7ATYjTIgM3jUlt4UM3IypQ    5.0   

                                                text                 date  \
0  If you decide to eat here, just be aware it is...  2018-07-07 22:09:11   
1  I've taken a lot of spin classes over the year...  2012-01-03 15:28:18   

  preprocessed_text  
0              None  
1              None  
Preprocessing completed! Saved the preprocessed Yelp sample.
